# Чтение данных

Качаем датасет (команды, перед которыми стоит восклицательный знак, выполняются в консоли)

In [1]:
# !wget http://files.deeppavlov.ai/datasets/snips_intents/train.csv

Читаем его библиотекой pandas

In [2]:
import pandas as pd

data = pd.read_csv('train.csv')

Посмотрим, как выглядят наши данные

In [3]:
data.head()

,text,intents
0,Add another song to the Cita RomГЎntica playli...,AddToPlaylist
1,add clem burke in my playlist Pre-Party R&B Jams,AddToPlaylist
2,Add Live from Aragon Ballroom to Trapeo,AddToPlaylist
3,add Unite and Win to my night out,AddToPlaylist
4,Add track to my Digster Future Hits,AddToPlaylist


Только тексты и интенты. Ничего лишнего. 

Сколько всего данных и как распределены интенты?

In [4]:
print(f'Всего данных: {data.shape[0]}')

data['intents'].value_counts()

Всего данных: 15884


GetWeather              2300
PlayMusic               2300
BookRestaurant          2273
SearchScreeningEvent    2259
RateBook                2256
SearchCreativeWork      2254
AddToPlaylist           2242
Name: intents, dtype: int64

преобразуем названия интентов в числа от 0 до 6 и положим эти метки классов в отдельный столбец target

In [5]:
data['target'] = data['intents'].astype('category').cat.codes

## Разбиваем датасет на обучающий и тестовый

Обычно задачи в машинном обучении поставлены как "обучиться на имеющихся данных, чтобы потом работать с новыми". 

Но конкретно сейчас у нас нет этих самых новых данных, поэтому мы разделим имеющийся датасет на две части. На одной будем учить модели, на другой -- проверять. 

*на самом деле, данные почти всегда делят на обучающую, валидационную и тестовую подвыборки, валидационную используют для промежуточной проверки модели и настройки параметров, которые невозможно настроить в ходе обучения модели (скажем, архитектура модели, критерии завершения итерационного алгоритма обучения, степень интерполяционного полинома, количество моделей в ансамбле и тд). Также существует перекрестная проверка (кросс-валидация), когда модель обучается и проверяется несколько раз на разном разбиении выборки на обучающую и валидационную (тестовую подвыборку при этом не трогают).           
Но в данном примере мы обойдемся очень простым подходом*

In [6]:
from sklearn.model_selection import train_test_split

data_train, data_test = train_test_split(data, random_state=42, test_size=0.2)

# Простое решение: tf-idf/logreg

Общая идея у всех подходов одна и та же: 

1. Мы каким-то образом преобразуем тексты в числовые вектора одинаковой длины
2. На получившейся выборке числовых векторов обучаем модель. 

Конкретно сейчас для преобразования текстов в вектора мы будем использовать tf-idf. Суть этого подхода вкратце:

* мы игнорируем порядок слов в предложении
* составляем словарь всех слов из всех предложений
* для каждого предложения считаем количество вхождений всех слов из словаря. Получим длинный вектор, в котором большинство значений будут нулями. Поэтому, кстати, для хранения итоговой матрицы объектов-признаков используется sparse-формат (разреженная матрица, когда мы просто перечисляем ненулевые элементы и их позиции, а не храним всю матрицу в памяти явно). 
* для каждого слова делим все его посчитанные вхождения на частоту встречаемости этого слова в различных текстах.

*Тут проще пояснить на примере.      
Допустим, мы векторизуем таким образом большое количество научных статей. У статей по математике слово "интеграл" будет встречаться довольно часто (а среди всех статей оно будет довольно редким), поэтому итоговый "вес" этого слова в векторах будет довольно большим. Аналогично, например, с биологией и словом "хромосома".    
А слова, которые встречаются часто в самых разных статьях (например, слово "следовательно" или какой-нибудь предлог) будут получать меньший "вес". Аналогично со словами, которые встречаются очень редко -- например, в какой-то статье предлагается новый метод, авторы которого его как-то назвали. Если статья новая, то это название будет встречаться только в ней (в векторе, соответствующем этой статье вес этого слова будет очень большим, во всех остальных -- нулевой)*

Алгоритм tf-idf очень легко реализуется на python, но мы сейчас этого делать не будем, используем готовую реализацию из scikit-learn. 

Но сперва данные нужно будет дополнительно предобработать, тк в них есть слова в разных регистрах (слова 'add' и 'Add' с точки зрения компьютера -- разные) и разных формах. 

Предобработку тут сделаем простую: 

1. приведение к нижнему регистру и лемматизация
2. удаление знаков препинания и стопслов 

## предобработка

In [7]:
# !python -m spacy download en_core_web_sm

In [8]:
import spacy
spacy_parser = spacy.load("en_core_web_sm")

def preprocess_text(text):
    tokenized_text = spacy_parser(text)
    
    processed_text = []
    for token in tokenized_text:
        if not token.is_stop and token.pos_ != 'PUNCT':
            processed_text.append(token.lemma_)
    
    return ' '.join(processed_text)

Предобработка текстов может длиться ощутимое время, поэтому сразу заводим прогресс-бар

In [9]:
from tqdm.notebook import tqdm
tqdm.pandas()  # включаем совместимость с pandas

In [10]:
data_test['processed_text'] = data_test['text'].progress_apply(preprocess_text)
data_train['processed_text'] = data_train['text'].progress_apply(preprocess_text)

/anaconda/envs/demo_intents/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


/anaconda/envs/demo_intents/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [11]:
data_train.head()

,text,intents,target,processed_text
10083,play the game Piety Street,SearchCreativeWork,5,play game piety street
4872,Is it chilly in Curacao,GetWeather,2,be chilly curacao
447,add Dean Martin track to metal xplorer playlist,AddToPlaylist,0,add dean martin track metal xplorer playlist
1791,Add another tune to my Soft Rock playlist.,AddToPlaylist,0,add tune soft rock playlist
4965,Is it going to get windy today in Tunisia?,GetWeather,2,be go windy today tunisia


## tf-idf векторизация

In [12]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer()

X_train = tfidf.fit_transform(data_train['processed_text'])
X_test = tfidf.transform(data_test['processed_text'])

In [13]:
X_train

<12707x10350 sparse matrix of type '<class 'numpy.float64'>'
	with 69723 stored elements in Compressed Sparse Row format>

## Logistric regression

Это алгоритм классификации (несмотря на слово "регрессия" в его названии). Он линейный, то есть, пытается разделить объекты в пространстве признаков с помощью гиперплоскостей.

Тут мы тоже воспользуемся готовой реализацией. 

In [14]:
y_train = data_train['target'].values
y_test = data_test['target'].values

In [15]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression(solver='liblinear', multi_class='ovr', random_state=42)

In [16]:
model.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr',
          n_jobs=None, penalty='l2', random_state=42, solver='liblinear',
          tol=0.0001, verbose=0, warm_start=False)

## Считаем качество на тесте

Метрик качества существует довольно много. Самая простая -- accuracy (просто доля правильных ответов). 

Эта метрика обладает серьезным недостатком: если бы у нас было 99% объектов первого класса и 1% объектов второго, и модель бы всегда предсказывала только первый класс (то есть, вела себя абсолютно неадекватно), то метрика accuracy была бы равно 0.99


Но с другой стороный, значения этой метрики проще всего интерпретировать. Да и у нас в выборке объектов разных классов приблизительно поровну, поэтому здесь использование метрики accuracy оправдано. 

In [17]:
y_pred = model.predict(X_test)

print(f'Accuracy = {(y_pred == y_test).mean()}')

Accuracy = 0.9694680516210261


### Accuracy = 0.969

# Более продвинутый вариант: ELMO

Далеко не всегда tf-idf себя так хорошо показывает. Да и точность 0.97 нас может по каким-то причинам не устроить. 

Хочется чего-то получше. Решение есть! Эмбеддинги. 

Эмбеддингами называют любые преобразования входных данных в числовые вектора посредством нейронных сетей. Самые простые эмбеддинги слов -- это Word2Vec. Они хорошо преобразуют слова в вектора (при этом смысл слов соотносится с операциями над векторами "король - мужчина + женщина = королева"), но не умеют учитывать контекст слов. Посколько в датасете есть много конструкций из нескольких слов, то лучше бы взять что-то, умеющее работать с контекстом. Например, ELMO.


Мы воспользуемся предобученной моделью от авторов ELMO и их же реализацией. 

## Запускаем ELMO

In [18]:
# !wget https://s3-us-west-2.amazonaws.com/allennlp/models/elmo/2x4096_512_2048cnn_2xhighway/elmo_2x4096_512_2048cnn_2xhighway_options.json
# !wget https://s3-us-west-2.amazonaws.com/allennlp/models/elmo/2x4096_512_2048cnn_2xhighway/elmo_2x4096_512_2048cnn_2xhighway_weights.hdf5

In [19]:
# Раскомментируйте эти строки, если не хотите использовать видеокарту для расчетов
# Если у вас нет возможности считать на видеокарте, то раскомментировать необязательно

# import os
# os.environ['CUDA_VISIBLE_DEVICES'] = ''

In [20]:
import torch

device = torch.device('cuda')  # если доступна видеокарта, то будем использовать ее, иначе -- считать на CPU

In [21]:
!ls

 elmo_2x4096_512_2048cnn_2xhighway_options.json
 elmo_2x4096_512_2048cnn_2xhighway_weights.hdf5
'intent classification (copy).ipynb'
'intent classification.ipynb'
 intents_conda_requirements.txt
 intents_pip_requirements.txt
 train.csv
 train.csv.1


In [22]:
from allennlp.modules.elmo import Elmo, batch_to_ids

elmo = Elmo(
    options_file='elmo_2x4096_512_2048cnn_2xhighway_options.json',
    weight_file='elmo_2x4096_512_2048cnn_2xhighway_weights.hdf5',
    num_output_representations=1
).to(device)

07/27/2020 21:17:59 - INFO - allennlp.modules.elmo -   Initializing ELMo


In [23]:
elmo(batch_to_ids(
    ['this is a test'.split(), 
     'another test sentence'.split()]).to(device)
    )

{'elmo_representations': [tensor([[[-0.0000, -0.0000,  0.0000,  ..., -0.0000,  0.0000, -0.0000],
           [-0.0000, -0.0000,  0.1639,  ..., -0.5736,  0.9779,  0.0000],
           [ 0.0399,  0.4843,  0.0000,  ...,  0.5934, -0.0000, -0.0640],
           [-1.9126,  0.0000,  0.0000,  ...,  0.4616,  0.3698,  0.0000]],
  
          [[ 0.7848, -0.0000,  0.7963,  ...,  0.0000,  0.0000, -0.1783],
           [-0.0000, -0.0000,  0.3812,  ...,  0.0000,  0.0000,  1.3101],
           [-0.4574,  0.0000,  0.0000,  ...,  0.0000, -0.1770,  0.0000],
           [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]]],
         device='cuda:0', grad_fn=<DropoutBackward>)],
 'mask': tensor([[1, 1, 1, 1],
         [1, 1, 1, 0]], device='cuda:0')}

Мы получаем эмбеддинги отдельно для каждого слова. А нам нужны эмбеддинги предложения в целом. 

Самый простой способ -- усреднить эмбеддинги слов. 

## Векторизуем тексты датасета

In [24]:
def get_text_embeddings(text):
    tokenized_text = [token.text for token in spacy_parser(text)]
    vectorized_text = batch_to_ids([tokenized_text]).to(device)
    with torch.no_grad():  # мы не обучаем ELMO, соответственно, указываем, что градиенты нам не нужны
        elmo_word_embeddings = elmo(vectorized_text)['elmo_representations'][0].squeeze().cpu().numpy()
    elmo_text_embeddings = elmo_word_embeddings.mean(axis=0)
    
    return elmo_text_embeddings

Получаем эмбеддинги фраз из датасета:

In [25]:
import numpy as np

def get_elmo_embeddings_for_dataset(df):
    texts = df['text'].values
    embeddings = []
    
    for text in tqdm(texts):
        elmo_text_embeddings = get_text_embeddings(text)
        embeddings.append(elmo_text_embeddings)
    return np.vstack(embeddings)

In [26]:
X_train_elmo = get_elmo_embeddings_for_dataset(data_train)
X_test_elmo = get_elmo_embeddings_for_dataset(data_test)

## Обучаем модель

In [27]:
model.fit(X_train_elmo, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr',
          n_jobs=None, penalty='l2', random_state=42, solver='liblinear',
          tol=0.0001, verbose=0, warm_start=False)

## Замеряем качество

In [28]:
y_pred = model.predict(X_test_elmo)

print(f'Accuracy = {(y_pred == y_test).mean()}')

Accuracy = 0.9779666351904313


### Accuracy = 0.978

Значение может немного отличаться, это особенности работы самой ELMO, связанные с сохранением скрытых состояний LSTM-слоёв и их перенос в следующие запуски. 